In [74]:
import bw2data as bd
import bw2calc as bc
import numpy as np
from tqdm import tqdm
from scipy.stats import spearmanr
import math

In [2]:
bd.projects

Brightway2 projects manager with 9 objects:
	LC IMPACT case study
	Sanofi
	USEEIO example
	barca 2022
	default
	michael question
	spreadsheet
	supply chain graph
	supply chain graph 3
Use `projects.report()` to get a report on all projects.

In [3]:
bd.projects.set_current("LC IMPACT case study")

In [6]:
gwp = ('LC-IMPACT', 'Climate Change', 'Human Health', 'Marginal', 'All', 'Infinite')

In [8]:
act = bd.Database("ecoinvent").random()
act

'market for chloroacetic acid' (kilogram, ('ecoinvent', 'RER'), None)

In [12]:
lca = bc.LCA({act: 1}, gwp, use_distributions=True)
lca.lci()
lca.lcia()
lca.score

2.7581676354144805e-05

In [16]:
lca.technosphere_mm.input_data_vector().shape

(241507,)

In [19]:
lca.technosphere_mm.input_row_col_indices().shape

(241507,)

In [54]:
def draw_samples(lca, samples=50):
    data = np.zeros((samples, len(lca.technosphere_mm.input_data_vector())))
    scores = np.zeros((samples,))
    
    for i, _ in tqdm(zip(range(samples), lca)):
        data[i, :] = lca.technosphere_mm.input_data_vector()
        scores[i] = lca.score
        
    return data, scores

In [55]:
samples, scores = draw_samples(lca)

50it [00:35,  1.39it/s]


In [108]:
def get_spearman_corr_coefficients(samples, scores, chunk_size=100):
    all_same_mask = (np.diff(samples.T).sum(axis=1) != 0).T
    samples_masked = samples[:, all_same_mask]
    chunks = math.ceil(samples_masked.shape[1] / chunk_size)

    corr = np.zeros_like(all_same_mask, dtype=float)
    corr_masked = corr[all_same_mask]
    
    for i in tqdm(range(chunks)):
        corr_masked[i * chunk_size:(i + 1) * chunk_size] = spearmanr(
            samples_masked[:, i * chunk_size:(i + 1) * chunk_size], 
            scores
        )[0][:-1, -1]
    
    corr[all_same_mask] = corr_masked
    return corr

In [109]:
corr = get_spearman_corr_coefficients(samples, scores)

100%|██████████| 1913/1913 [00:35<00:00, 53.40it/s]


In [129]:
def get_sensitive_exchanges(corr, lca, cutoff=25):
    top_elems = lca.technosphere_mm.input_row_col_indices()[np.argsort(corr)][:cutoff]
    top_elems_mapped = [(lca.dicts.product.reversed[x], lca.dicts.activity.reversed[y]) for x, y in top_elems]
    
    return [
        (
            score,
            lca.technosphere_matrix[indices[0], indices[1]],
            bd.get_node(id=row[0]),
            bd.get_node(id=row[1]),
            indices,
            row,
        )
        for row, indices, score in zip(top_elems_mapped, top_elems, sorted(corr))
    ]

In [131]:
get_sensitive_exchanges(corr, lca, 10)

[(-0.6358223289315726,
  -0.5945356848790568,
  'market for acetic acid, without water, in 98% solution state' (kilogram, GLO, None),
  'chloroacetic acid production' (kilogram, ('ecoinvent', 'RER'), None),
  (16120, 8109),
  (20548, 12537)),
 (-0.5893397358943577,
  -2.903356577719151e-05,
  'market for glyphosate' (kilogram, GLO, None),
  'orange production, fresh grade' (kilogram, ZA, None),
  (2725, 3390),
  (7153, 7818)),
 (-0.5684993997599038,
  -204234.5270029679,
  'market for waste asphalt' (kilogram, ('RoW', 'RoW_134'), None),
  'infrastructure construction, for regional distribution of oil product' (unit, ('RoW', 'RoW_213'), None),
  (7962, 16423),
  (12390, 20851)),
 (-0.5545738295318128,
  -1.4505546185755663e-05,
  'market for inorganic potassium fertiliser, as K2O' (kilogram, CA, None),
  'bamboo forestry, sustainable forest management' (kilogram, ('RoW', 'RoW_311'), None),
  (4362, 4670),
  (8790, 9098)),
 (-0.554093637454982,
  -1.2661377107051052e-08,
  'market for wa